In [1]:
import numpy
from scipy.linalg import solve

In [2]:
def generateMatrix(N, sigma):
    """Computes the matrix for the diffusion equation with Crank-Nicolson
    Dirichlet condition at i=0, Neumann at i = -1
    
    Parameters:
    ----------
    N: int
        Number of discretization points
    sigma: float
        alpha*dt/dx^2
        
    Returns: 
    ----------
    A: 2D numpy array of float
        Matrix for diffusion equation
    """
    
    #Setup the diagonal
    d = 2*numpy.diag(numpy.ones(N-2)*(1+1./sigma))
    
    #Consider Neumann BC
    d[-1, -1] = 1+2./sigma
    
    #Setup upper diagonal
    ud = numpy.diag(numpy.ones(N-3)*-1, 1)
    
    #Setup lower diagonal
    ld = numpy.diag(numpy.ones(N-3)*-1, -1)
    
    A = d + ud + ld
    
    return A